In [1]:
# installing Necessary packages

%pip install psycopg2-binary sqlalchemy pandas azure-storage-blob dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importing Necessary Libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
import os
from pathlib import Path

In [25]:
# Data Extraction

try:
    data = pd.read_csv(r'zipco_transaction.csv')
    print('Data Extracted Successfully')
except Exception as e:
    print(f"An error occured: {e}")

Data Extracted Successfully


In [4]:
data.head()

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [6]:
# Data Cleaning and Transformation
# Remove Duplicated entries

data.drop_duplicates(inplace=True)

In [7]:
# Handling Missing Data by filling numeric values with mean or median

numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_columns:
    data.fillna({col: data[col].mean()}, inplace=True)

In [8]:
# Handling Missing Data by filling categorical values with unknown
cate_columns = data.select_dtypes(include=['object']).columns
for col in cate_columns:
    data.fillna({col: 'Unknown'}, inplace=True)

In [9]:
# Changing the date type
data['Date'] = pd.to_datetime(data['Date'])

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   PaymentType             1000 non-null   object        
 6   PromotionApplied        1000 non-null   bool          
 7   Weather                 1000 non-null   object        
 8   Temperature             1000 non-null   float64       
 9   StaffPerformanceRating  1000 non-null   object        
 10  CustomerFeedback        1000 non-null   object        
 11  DeliveryTime_min        1000 non-null   int64         
 12  OrderType               1000 non-null   object        

In [11]:
# Create the product table
products = data[['ProductName']].drop_duplicates().reset_index(drop=True)
products.index.name = 'ProductID'
products = products.reset_index()

In [12]:
products.head(20)

,ProductID,ProductName
0,0,Vanilla Cake
1,1,Red Velvet Cake
2,2,Chocolate Cake
3,3,Carrot Cake
4,4,Pizza Pepperoni
5,5,Pizza Margherita
6,6,Pizza Beef Double decker
7,7,Cheesecake
8,8,Fruit Cake


In [13]:
# Create the staff table
staff = data[['Staff_Name','Staff_Email']].drop_duplicates().reset_index(drop=True)
staff.index.name = 'StaffID'
staff = staff.reset_index()

In [14]:
staff.head(10)

,StaffID,Staff_Name,Staff_Email
0,0,John Bridges,pdavidson@example.com
1,1,Sarah Bentley,ajohnson@example.net
2,2,Connie Cervantes,michele29@example.net
3,3,Jessica Stewart,xwilson@example.org
4,4,Cheryl Carpenter,christine96@example.org
5,5,Jessica Coleman,sandra03@example.org
6,6,Kelly Johnston,edwardsaaron@example.org
7,7,Rebecca George,michaelperez@example.org
8,8,Sara Aguilar,nicholasmullins@example.org
9,9,Michelle Pace,freemanjose@example.com


In [15]:
# Create the customers table
customers = data[['CustomerName','CustomerAddress','Customer_PhoneNumber','CustomerEmail']].drop_duplicates().reset_index(drop=True)
customers.index.name = 'CustomerID'
customers = customers.reset_index()

In [16]:
customers.head()

,CustomerID,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail
0,0,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net
1,1,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com
2,2,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com
3,3,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net
4,4,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net


In [17]:
data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [18]:
# Create the Transactions table
transaction = data.merge(products, on=['ProductName'], how='left') \
                  .merge(customers, on=['CustomerName','CustomerAddress','Customer_PhoneNumber','CustomerEmail'], how='left') \
                  .merge(staff, on=['Staff_Name','Staff_Email'], how='left') \

transaction.index.name = 'TransactionID'
transaction =  transaction.reset_index() \
                          [['Date','TransactionID','ProductID','Quantity', 'UnitPrice', 'StoreLocation', 'PaymentType', 'PromotionApplied', \
                                'Weather', 'Temperature', 'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min', 'OrderType', \
                                    'CustomerID', 'StaffID', 'DayOfWeek', 'TotalSales']]



In [19]:
# Save data as .csv files

data.to_csv(r'data\clean_data.csv', index=False)
customers.to_csv(r'data\customers.csv', index=False)
products.to_csv(r'data\products.csv', index=False)
staff.to_csv(r'data\staff.csv', index=False)
transaction.to_csv(r'data\transaction.csv', index=False)


In [20]:
# Data Loading
# Load the environment variables from the .env files
load_dotenv()

connect_str = os.getenv('AZURE_CONNECTION_STRING_VALUE')
container_name = os.getenv('CONTAINER_NAME')

In [21]:
# create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

# Load the data to Azure Blob Storage
files = [
    (data, 'rawdata/cleaned_zipco_data.csv'), 
    (products, 'cleaneddata/products.csv'),
    (customers,'cleaneddata/customers.csv'),
    (staff, 'cleaneddata/staff.csv'),
    (transaction, 'cleaneddata/transaction.csv')
]

for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f'{blob_name} loaded into Azure Blob storage')

rawdata/cleaned_zipco_data.csv loaded into Azure Blob storage
cleaneddata/products.csv loaded into Azure Blob storage
cleaneddata/customers.csv loaded into Azure Blob storage
cleaneddata/staff.csv loaded into Azure Blob storage
cleaneddata/transaction.csv loaded into Azure Blob storage
